In [4]:
import json
import os
import re

# 设定语言
LANGS = [
    'java',
    'python',
    'javascript',
    'typescript',
    'go'
]

# 创建目录
NECESSARY_DIRS = [f'dataset/{x}' for x in LANGS] \
    + [f'enre_out/{x}' for x in LANGS] \
    + [f'repo/{x}' for x in LANGS]

for d in NECESSARY_DIRS:
    os.makedirs(d, exist_ok=True)


In [14]:
# 提取各个数据集的仓库 URL，已有忽略

# 根据使用修改：存放 CoEdPilot 训练集的目录
base_folder = r"C:\Users\aaa\Desktop\CoEdPilot-final-phase\dataset&model\locator_data"

def commit_url_to_repo_url(commit_url):
    return re.search(r"(.*?)/commit/\w+$", commit_url)[1]

# 提取 base_folder 中 from_path 数据集的后缀为 lang_suffix 的文件的仓库地址，存放到 out_path
def extract_distinct_repo_url(from_path, to_path, lang_suffix):
    distinct_repo_url = set()
    with open(os.path.join(base_folder, from_path), 'r', encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            json_data = json.loads(line)
            if json_data["file_name"].endswith(f".{lang_suffix}"):
                distinct_repo_url.add(commit_url_to_repo_url(json_data["html_url"]))
    distinct_repo_url = list(distinct_repo_url)

    with open(to_path, "w+", encoding="utf-8") as f:
        json.dump(distinct_repo_url, f)

# 为指定 lang 提取仓库地址
def extract_distinct_repo_url_for_lang(lang, lang_suffix):
    extract_distinct_repo_url(f"{lang}/test.jsonl", f"repo_url/{lang}_repo_urls.json", lang_suffix)

In [15]:
extract_distinct_repo_url_for_lang("javascript", "js")

In [1]:
# 运行对应语言的 ENRE 程序，并制作数据集

import make_nn_dataset_for_lang
import importlib
importlib.reload(make_nn_dataset_for_lang)
import json

# 为指定语言运行 make_dataset
def make_dataset_for_lang(lang):
    repos = []
    with open(f"repo_url/{lang}_repo_urls.json", "r", encoding="utf-8") as f:
        repos = json.load(f)

    for repo_url in repos:
        make_nn_dataset_for_lang.make_dataset(repo_url, lang)

In [2]:
make_nn_dataset_for_lang.make_dataset("https://github.com/vercel/next.js", "javascript")

Ignoring https://github.com/vercel/next.js. Repo already cloned
Executing command: "cd "c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript" && node c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\typescript\enre-ts-0.0.1-gamma.js -i c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\repo\javascript\next.js -v"


In [5]:
make_dataset_for_lang("javascript")

Ignoring https://github.com/vuejs/vue-cli. Repo already cloned
Executing command: "cd "c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript" &&node c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript\enre-ts-0.0.1-gamma.js -i c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\repo\javascript\vue-cli -o vue-cli-report-enre.json"
ENRE javascript: generated dep for "vue-cli" at c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript\vue-cli-report-enre.json
ENRE cannot generated out file c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript\vue-cli-report-enre.json
Ignoring https://github.com/yarnpkg/yarn. Repo already cloned
Executing command: "cd "c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript" &&node c:\Users\aaa\Desktop\edit-pilot-model\dependency_analyzer\enre_out\javascript\enre-ts-0.0.1-gamma.js -i c:\Users\aaa\Desktop\edit-pilot-model\dependency_an

In [5]:
import os
import json
from run import main

DATASET_TYPES = ["test", "train", "valid"]

def combine_dataset_for_lang(lang):
    dataset_dir = f"dataset/{lang}"
    dataset_of_repos = [f.path for f in os.scandir(dataset_dir) if f.is_dir()]
    for dataset_type in DATASET_TYPES:
        all_data = []
        for repo in dataset_of_repos:
            with open(f"{repo}/{dataset_type}.json", "r", encoding="utf-8") as f:
                all_data += json.load(f)
        with open(os.path.join(dataset_dir, f"{dataset_type}.json"), "w") as f:
            json.dump(all_data, f)
        
def run_training_main(lang):
    model_name = 'huggingface/CodeBERTa-small-v1'
    batch_size = 32
    train = True
    test = True
    print(f'--model: {model_name}, --lang: {lang}, --train: {train}, --test {test}, --batch_size: {batch_size}')
    main(model_name, lang, batch_size, train, test)


In [6]:
combine_dataset_for_lang("java")

In [7]:
run_training_main("java")

--model: huggingface/CodeBERTa-small-v1, --lang: java, --train: True, --test True, --batch_size: 32


Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at huggingface/CodeBERTa-small-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You shoul

No checkpoint found. Training from scratch...


100%|██████████| 2138/2138 [08:24<00:00,  4.23it/s]


mode: dev, A depend on B
acc: 0.9698130308301782, precision: 0.964363916359874, recall: 0.985637696129064, F1: 0.9748847618005132
mode: dev, A depend by B
acc: 0.9673132866519508, precision: 0.96370441922241, recall: 0.9819635826771653, F1: 0.9727483241925655
Validation Loss: 0.10413794356012339


...

mode: dev, A depend on B
acc: 0.9746078617685324, precision: 0.9666482844606421, recall: 0.991490826816192, F1: 0.978911969308842
mode: dev, A depend by B
acc: 0.9740669814492668, precision: 0.9660862474216914, recall: 0.9911417322834646, F1: 0.9784536157601963
Validation Loss: 0.08887577650657358


100%|██████████| 4276/4276 [28:28<00:00,  2.50it/s]


mode: test, A depend on B
acc: 0.9747178692550579, precision: 0.9668389216993714, recall: 0.9917677378378709, F1: 0.9791446848791429
mode: test, A depend by B
acc: 0.974045433282657, precision: 0.9663763419080413, recall: 0.991091939977516, F1: 0.97857810756191
Test Loss: 0.08917729847261227


Training:   1%|          | 156/14965 [04:53<7:44:48,  1.88s/it, loss=0.0111] 


KeyboardInterrupt: 